#### Import packages

In [ ]:
# libraries
import pandas as pd
import pydbtools as pydb
import boto3
from datetime import datetime
#import re
import os  # for file paths
import awswrangler as wr

In [ ]:
# Structured  dataframes
pd.set_option("display.max_columns", 100)
pd.set_option("display.width", 900)
pd.set_option("display.max_colwidth", 200)

#### Assigning key variables

In [ ]:
# define some variables to be used throughout the notebook
db1 = "familyman_derived_dev_v2" #database where Familyman data is stored
db2 = "familyman_dev_v2"
snapshot_date = "2022-05-23" #To update where necessary
db3 = "fcsq" #database where tables created as part of FCSQ processing are stored where required

In [ ]:
#Select all disposal events*/
pydb.create_temp_table(
f""" 
SELECT 
  e.event,
  e.case_number,
  e.event_model,
  CASE WHEN e.event_model IN ('C43', 'ORDNOM', 'ORDREF','G63')
      THEN 'GEN'
      ELSE e.event_model END 
    AS lookup_event_code,
  e.receipt_date,
  e.entry_date,
  COALESCE(e.receipt_date, e.entry_date) AS disp_date,
  e.creating_court,
  CASE WHEN e.event_model = 'G63'
        THEN app.application_type_value_array
       WHEN e.order_type_value_if_in_fields_array IS NULL
        THEN ARRAY[NULL]
         ELSE e.order_type_value_if_in_fields_array END  
    AS all_event_ord_types
FROM 
  {db1}.events_derived AS e
  LEFT JOIN {db1}.events_derived AS app
    ON e.withdrawn_application_type_value = app.event
      AND e.mojap_snapshot_date = app.mojap_snapshot_date    
WHERE
  e.event_model IN ('C23','C25','C26','C27','C28','C29',
                    'C30','C31','C32A','C32B','C33','C34A','C34B','C35A','C35B','C37',
                    'C38A','C38B','C39','C40','C42','C43A','C44A','C44B','C45A','C45B',
                    'C53','C80','C81','C82',
                    
                    'C21', 'C24', 'C36','C43', 'ORDNOM', 'ORDREF',
                    
                    'G63')
  AND e.mojap_snapshot_date = DATE'{snapshot_date}'



""",

"ca_extract_disps_d")

In [ ]:
#split disposal events to one per row
pydb.create_temp_table(
f""" 
SELECT 
  t1.*,
  TRIM(ord_type) as order_type,
  CASE WHEN t2.order_code < 19 OR (SUBSTR(case_number,5,1) = 'C') THEN 'C'
        ELSE 'P' END 
      AS order_case_type,  
  CASE WHEN t1.event_model = 'G63' THEN 3
       WHEN t1.event_model = 'ORDREF' THEN 4
       WHEN t1.event_model = 'ORDNOM' THEN 5
       WHEN t2.order_code BETWEEN 40 AND 45 THEN 2
       ELSE 1 END
    AS disp_type_code,
  CASE WHEN t1.event_model = 'G63' THEN 'Withdrawn'
       WHEN t1.event_model = 'ORDREF' THEN 'Refused'
       WHEN t1.event_model = 'ORDNOM' THEN 'No order'
       WHEN t2.order_code BETWEEN 40 AND 45 THEN 'Interim Order'
       ELSE 'Order' END
    AS disp_type,
  order_code,
  order_desc
FROM 
  __temp__.ca_extract_disps_d AS t1
  CROSS JOIN UNNEST(all_event_ord_types) 
        AS t(ord_type)    
  LEFT JOIN {db3}.ca_order_type_lookup t2
    ON t1.lookup_event_code = t2.event_model
    AND TRIM(ord_type) = t2.value


""",

"ca_disps_split_d")